In [3]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import scipy as sp
import quandl
import functools
import seaborn as sns
import plotnine as p9
import math
import datetime
import statsmodels.formula.api as sm
import statsmodels
import statsmodels.api as smapi
from statsmodels.regression.rolling import RollingOLS
import hmmlearn
from hmmlearn.hmm import GaussianHMM
import warnings
import os
import matplotlib.dates as mdates
import random

In [4]:
def run_strategy(strat_a, strat_b):
    cards = []
    middle_cards = []
    a_cards = []
    b_cards = []
    for i in range(0,13):
        cards.append(i+1)
    a_score = 0
    b_score = 0
    middle_score = 0
    for i in range(0,13):
        middle_card = draw_card(cards)
        middle_score = middle_card
        a = strat_a(middle_card,middle_cards,b_cards,a_cards)
        b = strat_b(middle_card,middle_cards,a_cards,b_cards)
        if (a > b):
            a_score = a_score + middle_score
            middle_score = 0
        elif (b > a):
            b_score = b_score + middle_card
            middle_score = 0
        a_cards.append(a)
        b_cards.append(b)
        middle_cards.append(middle_card)
        cards = list_subtract(cards,[middle_card])
    return (a_score,b_score)
            
def get_winrate(strat_a,strat_b):
    a_wins = 0
    b_wins = 0
    ties = 0
    n = 1000
    a_avg = 0
    b_avg = 0
    for i in range(0,1000):
        (a,b) = run_strategy(strat_a,strat_b)
        a_avg = a_avg + a
        b_avg = b_avg + b
        if (b > a):
            b_wins = b_wins + 1
        elif (a > b):
            a_wins = a_wins + 1
        else:
            a_wins = a_wins + 0.5
    return a_wins/n
    
def base_winrate_table(names,list_of_strats):
    basic_strats = [det_0,det_1,learning,random_strat]
    basic_names = ['Matching','Upcard + 1', 'Basic Learning', 'Random']
    df = pd.DataFrame(index = range(0,len(names)))
    for i in basic_names:
        df[i] = np.zeros(len(names))
    for i in range(0,len(names)):
        for j in range(0,4):
            df[basic_names[j]][i] = get_winrate(list_of_strats[i],basic_strats[j])
            
    df.index = names

    return df
        
        
def winrate_table(strats,strat_functions):
    df = pd.DataFrame(index = range(0,len(strats)))
    for i in strats:
        df[i] = np.zeros(len(strats))

    for i in range(0,len(strats)):
        for j in range(0,len(strat_functions)):
            df[strats[i]][j] = get_winrate(strat_functions[j],strat_functions[i])
    df.index = strats
    return df
    
def draw_card(cards):
    return cards[random.randint(0,len(cards) -1)]
    
def list_subtract(x,y):
    return [item for item in x if item not in y]  

def minus(x,y):
    return list(pd.Series(x,dtype = 'int') - pd.Series(y,dtype = 'int'))

In [82]:
#Basic Strategies 
def deterministic(current_card,middle_cards,enemy_cards,player_cards, n):
    cards = []
    for i in range(0,13):
        cards.append(i+1)
    hand = list_subtract(cards,player_cards)
    deck = list_subtract(cards,middle_cards)
    if(current_card + n <= 13) and (current_card + n in hand):
        return current_card + n
    elif (current_card + n > 13):
        return hand[0]
    else:
        return hand[deck.index(current_card)]
            

def det_1(current_card,middle_cards,enemy_cards,player_cards):
    res = deterministic(current_card,middle_cards,enemy_cards,player_cards,1)
    #print(current_card,res)
    return res

def det_4(current_card,middle_cards,enemy_cards,player_cards):
    res = deterministic(current_card,middle_cards,enemy_cards,player_cards,4)
    #print(current_card,res)
    return res

def det_7(current_card,middle_cards,enemy_cards,player_cards):
    res = deterministic(current_card,middle_cards,enemy_cards,player_cards,7)
    #print(current_card,res)
    return res



def det_0(current_card,middle_cards,enemy_cards,player_cards):
    return deterministic(current_card,middle_cards,enemy_cards,player_cards,0)
    
def learning(current_card,middle_cards,enemy_cards,player_cards):
    if(len(middle_cards) > 0):
        n_est = enemy_cards[-1] - middle_cards[-1] 
        #print(n_est,current_card)
        if(n_est < 0):
            n_est = -1
        return deterministic(current_card,middle_cards,enemy_cards,player_cards,n_est + 1)
    else:
        cards = []
        for i in range(0,13):
            cards.append(i+1)
        hand = list_subtract(cards,player_cards)
        return hand[0]
    
def random_strat(current_card,middle_cards,enemy_cards,player_cards):
    cards = []
    for i in range(0,13):
        cards.append(i+1)
    avail_cards = list_subtract(cards,player_cards)
    return draw_card(avail_cards)

In [84]:
strats = ['Matching Strategy','Upcard + 1','Upcard + 4',"Upcard + 7",'Learning','Random']
strat_functions = [det_0,det_1,det_4,det_7,learning,random_strat]
df = winrate_table(strats,strat_functions)
df

,Matching Strategy,Upcard + 1,Upcard + 4,Upcard + 7,Learning,Random
Matching Strategy,0.500,0.000,1.0000,1.000,0.0000,0.9655
Upcard + 1,1.000,0.500,0.0000,1.000,0.0020,0.8795
Upcard + 4,0.000,1.000,0.5000,0.040,0.0340,0.2475
Upcard + 7,0.000,0.000,0.9680,0.500,0.0235,0.1105
Learning,1.000,1.000,0.9565,0.980,0.5000,0.6500
Random,0.042,0.128,0.6990,0.849,0.3490,0.5060


so we see random loses to matching which loses to upcard + 1 which both lose to learning. But we see the learning strategy perform poorly against random. This indicates that there is no Nash equilibream using basic strategies, since no matter what you are always inclined to switch to another strategy. So we now try to develop a "Good" strategy by taking the good aspects of basic strategies and combining them into a hybrid strategy. 

We design a more advanced "hybrid" strategy that is a combination of deterministic and random. This strategy plays like determinisitc, but  $n \in \mathbb{Z}\cap [-1,2]$ and n is picked randomly. If n = -1, the player will instead play the lowest card in their hand. If the card the player wants to play is not in their hand, they will also play the lowest card in their hand. 

In [93]:
#advanced strategies

def determ_random(current_card,middle_cards,enemy_cards,player_cards):
    n = random.randint(-1,1)
    if n == -1:
        cards = []
        for i in range(0,13):
            cards.append(i+1)
        avail_cards = list_subtract(cards,player_cards)
        return avail_cards[0]
    else:
        return deterministic(current_card,middle_cards,enemy_cards,player_cards,n * 2)
 
#looks at enemy performance over multiple turns instead of most recent
def robust_learning(current_card,middle_cards,enemy_cards,player_cards):
    residuals = minus(enemy_cards,middle_cards)
    if len(residuals) > 0:
        total = 0
        n = 0
        for residual in residuals:
            if(abs(residual) < 3):
                total = total + residual
                n = n + 1
        if(n > 0):
            mean = total/n
            return deterministic(current_card,middle_cards,enemy_cards,player_cards, mean + 1)
        
    return random_strat(current_card,middle_cards,enemy_cards,player_cards)

def mean_learning(current_card,middle_cards,enemy_cards,player_cards):
    if len(middle_cards) > 0:
        residuals = minus(enemy_cards,middle_cards)
        mean = pd.Series(residuals).mean()
        return deterministic(current_card,middle_cards,enemy_cards,player_cards, mean + 1) 
    else:
        return det_0(current_card,middle_cards,enemy_cards,player_cards)

        
#this assumes opponent is either deterministic or random. 
def pearson_strategy(current_card,middle_cards,enemy_cards,player_cards):
    if len(middle_cards) > 2:
        correlation = pd.Series(middle_cards).corr(pd.Series(enemy_cards))
        if (correlation < 0.2):
            return det_0(current_card,middle_cards,enemy_cards,player_cards) #assumes random and plays matching
        else:
            return mean_learning(current_card,middle_cards,enemy_cards,player_cards)
    else: 
        return learning(current_card,middle_cards,enemy_cards,player_cards)
        
    
def greedy(current_card,middle_cards,enemy_cards,player_cards):
    cards = []
    for i in range(0,13):
        cards.append(i+1)
    hand = list_subtract(cards,player_cards)
    deck = list_subtract(cards,middle_cards)
    if(current_card > 8):
        return hand[-1]
    else:
        return hand[0]

def counter_determ(current_card,middle_cards,hero_cards,n):
    cards = []
    for i in range(0,13):
        cards.append(i+1)
    hand = list_subtract(cards,hero_cards)
    deck = list_subtract(cards,middle_cards)
    if(current_card + n <= 13) and (current_card + n in hand):
        return current_card + n
    elif (current_card + n > 13):
        return hand[min(len(hand) -1,1)]
    else:
        return hand[min(deck.index(current_card) + 1,len(deck) -1)]    
    
def learning_counter(current_card,middle_cards,villian_cards,hero_cards):
    #check if learning strategy

    if(len(hero_cards) > 0):
        return counter_determ(current_card,middle_cards,hero_cards,hero_cards[-1] - middle_cards[-1] + 2)
    return 1


def detect_learning(middle_cards,villian_cards,hero_cards):
    total = 0
    for i in range(1,len(middle_cards)):
        if villian_cards[i] - middle_cards[i] == 1 + hero_cards[i -1] - middle_cards[i-1]:
            total = total + 1
    if len(middle_cards) > 0 and total/len(middle_cards) > 0.3:
        return True
    return False
        
def hybrid(current_card,middle_cards,villian_cards,hero_cards):
    if detect_learning(middle_cards,villian_cards,hero_cards):
        return learning_counter(current_card,middle_cards,villian_cards,hero_cards)
    else:
        return pearson_strategy(current_card,middle_cards,villian_cards,hero_cards)
            
    

In [99]:
strategy = ["Hybrid"]
function = [hybrid]
df = base_winrate_table(strategy,function)
df


,Matching,Upcard + 1,Basic Learning,Random
Hybrid,0.9995,0.858,0.852,0.815


In [87]:
strats = [ "Deterministic Random", "Robust Learning", "Pearson","Greedy","Learning Counter","Hybrid"]
strat_functions = [determ_random, robust_learning,pearson_strategy,greedy,learning_counter,hybrid]

df = base_winrate_table(strats,strat_functions)
df

,Matching,Upcard + 1,Basic Learning,Random
Deterministic Random,0.4730,0.4060,0.3985,0.6820
Robust Learning,0.9990,0.9970,0.2980,0.7020
Pearson,1.0000,0.8780,0.3125,0.8275
Greedy,0.1225,0.1020,0.3565,0.9055
Learning Counter,0.1585,0.5210,0.9315,0.5120
Hybrid,1.0000,0.8725,0.8490,0.8010


As we can see Deterministic random has a positive winrate into all of the previously mentioned strategies. Suprisingly 

In [100]:
strats =["Matching", "Basic Learning","Deterministic Random", "Robust Learning", "Correlation","Learning Counter","Hybrid"]
strat_functions = [det_0,learning,determ_random, robust_learning,pearson_strategy,learning_counter,hybrid]
df = winrate_table(strats,strat_functions)
df

,Matching,Basic Learning,Deterministic Random,Robust Learning,Correlation,Learning Counter,Hybrid
Matching,0.5000,0.0000,0.5305,0.0010,0.0000,0.8200,0.0020
Basic Learning,1.0000,0.5000,0.5630,0.6970,0.6675,0.0875,0.1405
Deterministic Random,0.4595,0.4305,0.5140,0.5065,0.3735,0.3575,0.3880
Robust Learning,1.0000,0.3330,0.4725,0.5195,0.5050,0.2580,0.4370
Correlation,1.0000,0.3140,0.6340,0.4965,0.5000,0.5495,0.5265
Learning Counter,0.1610,0.9190,0.6060,0.7710,0.4535,0.5000,0.4680
Hybrid,0.9990,0.8440,0.6280,0.5565,0.4935,0.5835,0.5000
